# Assignment V: GitHub and the ticketmaster.com API

## General

GitHub repository: https://github.com/Borschu/Assignment_05

Import packages:

In [18]:
import numpy as np
import pandas as pd
import requests
import time

## Setting up a new GitHub repository

see link above

## Getting to know the API

### (5)

Rate limits:
- 5000 API calls per day
- 5 requests per second
- you can only retrieve the 1000 th item (size * page < 1000)


### (6)

In [22]:
# create connection to API key
#with open(r"C:\Users\schur\OneDrive\Dokumente\Studium\Data Science Master Tübingen\1. Semester\Data Science Project Management\Assignment\key_assignment_5_python.py") as script:
#    exec(script.readline())

with open("C:/Users/schur/OneDrive/Dokumente/Studium/Data Science Master Tübingen/1. Semester/Data Science Project Management/Assignment/key_assignment_5_python.py") as script:
  exec(script.readline())


## Interacting with the API - the basics

### (7)

In [21]:
# create content for venue = Germany
con_ven_ger = requests.get("https://app.ticketmaster.com/discovery/v2/venues",
                          params = {"apikey": ticketmaster_key, "countryCode": "DE"}).json()

# print content
print(con_ven_ger)

{'_embedded': {'venues': [{'name': 'Gruenspan', 'type': 'venue', 'id': 'KovZpZAneakA', 'test': False, 'url': 'http://www.ticketmaster.de/venue/287155', 'locale': 'en-de', 'images': [{'ratio': '16_9', 'url': 'https://s1.ticketm.net/dbimages/2057v.', 'width': 205, 'height': 115, 'fallback': False}], 'postalCode': '22767', 'timezone': 'Europe/Berlin', 'city': {'name': 'Hamburg'}, 'country': {'name': 'Germany', 'countryCode': 'DE'}, 'address': {'line1': 'Grosse Freiheit 58'}, 'location': {'longitude': '9.958075', 'latitude': '53.551885'}, 'markets': [{'name': 'Germany', 'id': '210'}], 'dmas': [{'id': 610}], 'boxOfficeInfo': {'phoneNumberDetail': 'Gruenspan Große Freiheit 58 22767 Hamburg Tel: 040-313616 mail: info@gruenspan.de web: www.gruenspan.de'}, 'upcomingEvents': {'_total': 2, 'mfx-de': 2, '_filtered': 0}, 'ada': {'adaPhones': '+49.(0)1805 - 969 0000 (14 Ct./Min.)', 'adaCustomCopy': 'Sollten Sie besondere Sitzplatzwünsche haben, prüfen Sie bitte die Informationsseite des Veranstaltun

### (8)

In [71]:
# define size of page
size = con_ven_ger['page']['size']

# create empty vectors for all characteristics
name = [None]*size
city = [None]*size
postalCode = [None]*size
address = [None]*size
url = [None]*size
longitude = [None]*size
latitude = [None]*size

# for loop over all list entries for every characteristic of the venue
for i in range(size):
    # If a list element doesnt exist a key error occurs and the loop stops.
    # To avoid this the try-except-structure is built
    # if a KeyError occurs the value is codes as NaN
    try:
        name[i] = con_ven_ger['_embedded']['venues'][i]["name"]
    except KeyError:
        name[i] = np.nan
    try:
        city[i] = con_ven_ger['_embedded']['venues'][i]["city"]['name']
    except KeyError:
        city[i] = np.nan
    try:
        postalCode[i] = con_ven_ger['_embedded']['venues'][i]["postalCode"]
    except KeyError:
        postalCode[i] = np.nan
    try:
        address[i] = con_ven_ger['_embedded']['venues'][i]["address"]['line1']
    except KeyError:
        address[i] = np.nan
    try:
        url[i] = con_ven_ger['_embedded']['venues'][i]["url"]
    except KeyError:
        url[i] = np.nan
    try:
        longitude[i] = con_ven_ger['_embedded']['venues'][i]["location"]["longitude"]
    except KeyError:
        longitude[i] = np.nan
    try:
        latitude[i] = con_ven_ger['_embedded']['venues'][i]["location"]["latitude"]
    except KeyError:
        latitude[i] = np.nan
    
# create dataframe based on the previous created lists
df_venue_ger = pd.DataFrame({"name": name,
                           "city": city,
                           "postalCode": postalCode,
                           "address": address,
                           "url": url,
                           "longitude": longitude,
                           "latitude": latitude})

# print dataframe
print(df_venue_ger)


                             name             city postalCode  \
0                       Gruenspan          Hamburg      22767   
1               Huxleys Neue Welt           Berlin      10967   
2                   Virtual Event        Worldwide        NaN   
3           Ev. St. Jacobi Kirche     Sangerhausen      06526   
4             Evangelische Kirche           Senden      48308   
5                       HDI Arena         Hannover        NaN   
6                         Arsenal           Berlin      10785   
7       Freilichtbühne Heppenheim       Heppenheim      64646   
8                Schlosswallhalle        Osnabrück      49074   
9                Metropol Theater           Vechta      49377   
10        RÖDERMARK - Kulturhalle        Rödermark      63322   
11                           3922          Tambach      96479   
12                Nibelungenhalle         Walldürn      74731   
13            St. Johannis Kirche         Würzburg      97070   
14                   film

In [42]:
range(20)

range(0, 20)